# Ant Colony Optimization Algorithm for Sequential Ordering Problem


In [4]:
import os
import random
import time
import numpy as np
from collections import defaultdict
from pathlib import Path
import multiprocessing
import pandas as pd
import matplotlib.pyplot as plt


## The Sequential Ordering Problem


In [ ]:
class SOP:
    def __init__(self, file_path):
        self.file_path = file_path
        self.file_name = Path(file_path).name
        self.nodes, self.costs, self.precedence = self.parse_file(file_path)
        self.size = len(self.nodes)

        self.before = defaultdict(set)
        self.after = defaultdict(set)
        self.build_precedence_graph()

    def parse_file(self, file_path):
        with open(file_path, 'r') as f:
            lines = f.readlines()

        dimension = next(int(line.strip().split(':')[1]) 
                         for line in lines if line.startswith('DIMENSION'))
        
        matrix_start = next(i for i, line in enumerate(lines) 
                           if line.startswith('EDGE_WEIGHT_SECTION')) + 1
        
        if lines[matrix_start].strip().isdigit():
            matrix_start += 1
        
        matrix_data = []
        for i in range(matrix_start, matrix_start + dimension):
            row = list(map(int, lines[i].strip().split()))
            matrix_data.append(row)
        
        costs_matrix = np.array(matrix_data)
        
        nodes = list(range(1, dimension + 1))
        costs = {}
        for i in range(dimension):
            for j in range(dimension):
                if costs_matrix[i, j] != -1:
                    costs[(i+1, j+1)] = costs_matrix[i, j]
        
        precedence = []
        for i in range(dimension):
            for j in range(dimension):
                if costs_matrix[i, j] == -1 and i != j:
                    precedence.append((j+1, i+1))
        
        return nodes, costs, precedence

    def build_precedence_graph(self):
        for u, v in self.precedence:
            self.before[v].add(u)
            self.after[u].add(v)
        
        changed = True
        while changed:
            changed = False
            for node in self.nodes:
                old_before = self.before[node].copy()
                for b in old_before:
                    self.before[node].update(self.before[b])
                if len(self.before[node]) > len(old_before):
                    changed = True
                    
                old_after = self.after[node].copy()
                for a in old_after:
                    self.after[node].update(self.after[a])
                if len(self.after[node]) > len(old_after):
                    changed = True

    def is_valid(self, path):
        position = {node: i for i, node in enumerate(path)}
        
        for i, node in enumerate(path):
            for before_node in self.before[node]:
                if before_node in position and position[before_node] >= i:
                    return False
                
        for i in range(len(path) - 1):
            if (path[i], path[i+1]) not in self.costs:
                return False
                
        return True

    def total_cost(self, path):
        return sum(self.costs.get((path[i], path[i+1]), float('inf')) 
                  for i in range(len(path) - 1))

# Ant Colony Optimization Implementation


In [ ]:
class AntColonySolver:
    def __init__(self, sop, ants=50, iterations=100, 
                 alpha=1.0, beta=2.5, rho=0.1, q0=0.9,
                 initial_pheromone=0.1, min_pheromone_factor=0.001, max_pheromone_factor=5.0,
                 local_decay=0.1, max_stagnation=25,
                 candidate_list_size=20, elite_ants=5):
        
        self.sop = sop
        self.ants = ants
        self.iterations = iterations
        self.alpha = alpha  
        self.beta = beta    
        self.rho = rho      
        self.q0 = q0        
        self.initial_pheromone = initial_pheromone
        self.local_decay = local_decay
        self.max_stagnation = max_stagnation
        self.candidate_list_size = candidate_list_size  
        self.elite_ants = elite_ants  
        
        
        self.current_q0 = q0
        self.current_alpha = alpha
        self.current_beta = beta
        
        
        self.min_pheromone_factor = min_pheromone_factor
        self.max_pheromone_factor = max_pheromone_factor
        self.min_pheromone = None  
        self.max_pheromone = None  
        
        
        self.pheromone = {}
        for i in self.sop.nodes:
            for j in self.sop.nodes:
                if (i, j) in self.sop.costs:
                    self.pheromone[(i, j)] = initial_pheromone
        
        
        self.candidate_lists = self.precompute_candidate_lists()
        
        
        self.heuristic = {}
        for (i, j), cost in self.sop.costs.items():
            
            precedence_factor = 1.0
            if j in self.sop.after[i]:
                
                precedence_factor = 2.0
            
            
            base_heuristic = 1.0 / cost if cost > 0 else 0.0
            
            
            self.heuristic[(i, j)] = base_heuristic * precedence_factor
        
        
        self.best_solution = None
        self.best_cost = float('inf')
        self.last_improvement = 0
        self.best_solutions_history = []  
        self.iteration_stats = []
        self.convergence_factor = 0.0
        
        
        self.local_search_frequency = 0.2  
        
        
        self.update_pheromone_bounds()
    
    def precompute_candidate_lists(self):

        candidate_lists = {}
        
        for i in self.sop.nodes:
            
            possible_destinations = [(j, self.sop.costs.get((i, j), float('inf'))) 
                                     for j in self.sop.nodes 
                                     if (i, j) in self.sop.costs and i != j]
            
            
            possible_destinations.sort(key=lambda x: x[1])
            
            
            candidate_lists[i] = [j for j, _ in possible_destinations[:self.candidate_list_size]]
            
        return candidate_lists
    
    def update_pheromone_bounds(self):

        if self.best_cost < float('inf'):
            
            self.max_pheromone = self.max_pheromone_factor / self.best_cost
            self.min_pheromone = self.max_pheromone * self.min_pheromone_factor
        else:
            
            self.max_pheromone = 1.0
            self.min_pheromone = 0.001
    
    def construct_solution(self, use_candidate_lists=True):

        start_candidates = [n for n in self.sop.nodes if not self.sop.before[n]]
        start_node = start_candidates[0] if start_candidates else 1
        
        
        current_path = [start_node]
        unvisited = set(self.sop.nodes) - {start_node}
        current_node = start_node
        
        
        while unvisited:
            
            feasible_nodes = [node for node in unvisited 
                             if all(pred not in unvisited for pred in self.sop.before[node])
                             and (current_node, node) in self.sop.costs]
            
            if not feasible_nodes:
                
                return self.repair_solution(current_path)
            
            
            if use_candidate_lists and feasible_nodes:
                
                candidates = [n for n in self.candidate_lists.get(current_node, []) 
                             if n in feasible_nodes]
                
                
                if len(candidates) < min(3, len(feasible_nodes)):
                    candidates = feasible_nodes
            else:
                candidates = feasible_nodes
            
            
            next_node = self.select_next_node(current_node, candidates)
            
            
            self.local_pheromone_update(current_node, next_node)
            
            
            current_path.append(next_node)
            unvisited.remove(next_node)
            current_node = next_node
        
        
        if random.random() < self.local_search_frequency:
            current_path = self.local_search(current_path)
            
        
        if not self.sop.is_valid(current_path):
            current_path = self.repair_solution(current_path)
            
        return current_path
    
    def select_next_node(self, current_node, feasible_nodes):

        if not feasible_nodes:
            return None
            
        
        if random.random() < self.current_q0:
            
            return max(feasible_nodes, 
                      key=lambda j: self.pheromone.get((current_node, j), 0) ** self.current_alpha * 
                                    self.heuristic.get((current_node, j), 0) ** self.current_beta)
        else:
            
            total = 0.0
            probabilities = []
            
            for node in feasible_nodes:
                pheromone = self.pheromone.get((current_node, node), 0)
                heuristic_value = self.heuristic.get((current_node, node), 0)
                
                
                prob = (pheromone ** self.current_alpha) * (heuristic_value ** self.current_beta)
                probabilities.append(prob)
                total += prob
            
            if total == 0:
                
                return random.choice(feasible_nodes)
            
            
            probabilities = [p/total for p in probabilities]
            
            
            return random.choices(feasible_nodes, weights=probabilities, k=1)[0]
    
    def local_pheromone_update(self, i, j):

        if (i, j) in self.pheromone:
            self.pheromone[(i, j)] = (1 - self.local_decay) * self.pheromone[(i, j)] + \
                                     self.local_decay * self.initial_pheromone
    
    def global_pheromone_update(self, solutions_and_costs):

        for key in self.pheromone:
            self.pheromone[key] *= (1 - self.rho)
        
        
        for rank, (path, cost) in enumerate(solutions_and_costs):
            
            weight = max(0, self.elite_ants - rank) / self.elite_ants
            
            if weight > 0:
                
                delta = weight * (1.0 / cost) if cost > 0 else 0
                
                
                for i in range(len(path) - 1):
                    edge = (path[i], path[i+1])
                    if edge in self.pheromone:
                        self.pheromone[edge] += self.rho * delta
        
        
        for key in self.pheromone:
            self.pheromone[key] = min(self.max_pheromone, 
                                     max(self.min_pheromone, self.pheromone[key]))
    
    def repair_solution(self, path):

        if not path:
            return None
            
        
        valid_path = []
        remaining = set(path)
        
        while remaining:
            
            valid_candidates = [n for n in remaining 
                              if all(pred in valid_path or pred not in remaining 
                                    for pred in self.sop.before[n])]
            
            if not valid_candidates:
                
                return None
            
            
            next_node = min(valid_candidates, key=lambda n: path.index(n))
            valid_path.append(next_node)
            remaining.remove(next_node)
        
        return valid_path
    
    def local_search(self, solution):

        if not solution:
            return solution
        
        improved = True
        current_solution = solution.copy()
        current_cost = self.sop.total_cost(current_solution)
        
        
        
        max_attempts = min(30, len(solution))
        attempts = 0
        
        while improved and attempts < max_attempts:
            improved = False
            
            
            for i in range(len(current_solution) - 1):
                
                for j in range(i + 1, min(i + 5, len(current_solution))):
                    
                    new_solution = current_solution.copy()
                    new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
                    
                    
                    if self.sop.is_valid(new_solution):
                        new_cost = self.sop.total_cost(new_solution)
                        if new_cost < current_cost:
                            current_solution = new_solution
                            current_cost = new_cost
                            improved = True
                            break
                
                if improved:
                    break
            
            attempts += 1
        
        return current_solution
    
    def update_parameters(self, iteration, total_iterations):

        progress = iteration / total_iterations
        
        
        if len(self.iteration_stats) >= 10:
            recent_stats = self.iteration_stats[-10:]
            recent_costs = [stat['best_cost'] for stat in recent_stats]
            cost_stdev = np.std(recent_costs) if len(recent_costs) > 1 else 0
            
            
            self.convergence_factor = 0 if self.best_cost == 0 else cost_stdev / self.best_cost
        else:
            self.convergence_factor = 1.0  
        
        
        if self.convergence_factor < 0.01:
            
            
            self.current_q0 = max(0.5, self.q0 * 0.9)
            
            
            self.current_alpha = max(0.5, self.alpha * 0.9)
            
            
            self.current_beta = min(5.0, self.beta * 1.1)
        else:
            
            
            self.current_q0 = self.q0 * (1 - progress) + 0.95 * progress
            self.current_alpha = self.alpha
            self.current_beta = self.beta
    
    def diversify_search(self):
        
        strategy = random.choice(['reset', 'smoothing', 'invert'])
        
        if strategy == 'reset':
            
            for key in self.pheromone:
                
                self.pheromone[key] = self.initial_pheromone * random.uniform(0.8, 1.2)
                
        elif strategy == 'smoothing':
            
            avg_pheromone = sum(self.pheromone.values()) / len(self.pheromone)
            
            for key in self.pheromone:
                
                self.pheromone[key] = 0.2 * self.pheromone[key] + 0.8 * avg_pheromone
                
        elif strategy == 'invert':
            
            max_pheromone = max(self.pheromone.values())
            min_pheromone = min(self.pheromone.values())
            
            
            if max_pheromone > min_pheromone:
                for key in self.pheromone:
                    
                    normalized = (self.pheromone[key] - min_pheromone) / (max_pheromone - min_pheromone)
                    inverted = 1 - normalized
                    self.pheromone[key] = min_pheromone + inverted * (max_pheromone - min_pheromone)
        
        
        for key in self.pheromone:
            self.pheromone[key] = min(self.max_pheromone, 
                                     max(self.min_pheromone, self.pheromone[key]))
    
    def solve(self, verbose=True):

        start_time = time.time()
        stagnation_counter = 0
        best_iteration = 0
        
        
        solution_archive = []
        
        for it in range(self.iterations):
            
            self.update_parameters(it, self.iterations)
            
            
            self.update_pheromone_bounds()
            
            
            iteration_solutions = []
            
            
            for ant in range(self.ants):
                
                use_candidates = (ant % 2 == 0)
                
                
                path = self.construct_solution(use_candidate_lists=use_candidates)
                
                if path:
                    cost = self.sop.total_cost(path)
                    iteration_solutions.append((path, cost))
            
            
            iteration_solutions.sort(key=lambda x: x[1])
            
            
            if iteration_solutions and iteration_solutions[0][1] < self.best_cost:
                self.best_cost = iteration_solutions[0][1]
                self.best_solution = iteration_solutions[0][0].copy()
                best_iteration = it
                self.best_solutions_history.append((self.best_solution.copy(), self.best_cost))
                stagnation_counter = 0
            else:
                stagnation_counter += 1
            
            
            solution_archive.extend(iteration_solutions)
            solution_archive.sort(key=lambda x: x[1])
            solution_archive = solution_archive[:self.elite_ants * 2]  
            
            
            
            update_solutions = iteration_solutions[:self.elite_ants]
            if random.random() < 0.2:  
                update_solutions = solution_archive[:self.elite_ants]
                
            self.global_pheromone_update(update_solutions)
            
            
            if stagnation_counter >= self.max_stagnation:
                if verbose:
                    print(f"Stagnation detected at iteration {it} - diversifying search")
                self.diversify_search()
                stagnation_counter = 0
                
                
                self.current_q0 = self.q0
                self.current_alpha = self.alpha
                self.current_beta = self.beta
            
            
            self.iteration_stats.append({
                'iteration': it,
                'best_cost': self.best_cost,
                'iteration_best': iteration_solutions[0][1] if iteration_solutions else float('inf'),
                'avg_pheromone': sum(self.pheromone.values()) / len(self.pheromone) if self.pheromone else 0,
                'alpha': self.current_alpha,
                'beta': self.current_beta,
                'q0': self.current_q0,
                'convergence_factor': self.convergence_factor
            })
            
            
            if it > self.iterations / 2 and it - best_iteration > self.max_stagnation * 2:
                if verbose:
                    print(f"Early stopping at iteration {it}: no improvement for {it - best_iteration} iterations")
                break
            
            if verbose and (it % 10 == 0 or it == self.iterations - 1):
                print(f"Iteration {it}/{self.iterations}, Best cost: {self.best_cost}, " +
                     f"Alpha: {self.current_alpha:.2f}, Beta: {self.current_beta:.2f}, Q0: {self.current_q0:.2f}")
        
        
        if self.best_solution:
            improved_solution = self.extended_local_search(self.best_solution)
            improved_cost = self.sop.total_cost(improved_solution)
            
            if improved_cost < self.best_cost:
                self.best_cost = improved_cost
                self.best_solution = improved_solution
                
                if verbose:
                    print(f"Final improvement phase reduced cost from {self.best_cost} to {improved_cost}")
        
        end_time = time.time()
        
        if verbose:
            print(f"\nSolution for {self.sop.file_name}:")
            print(f"Path: {self.best_solution}")
            print(f"Cost: {self.best_cost}")
            print(f"Time: {end_time - start_time:.2f} seconds")
        
        return {
            'instance': self.sop.file_name,
            'solution': self.best_solution,
            'cost': self.best_cost,
            'time': end_time - start_time,
            'iterations': len(self.iteration_stats),
            'stats': self.iteration_stats
        }
    
    def extended_local_search(self, solution):

        if not solution:
            return solution
        
        best_solution = solution.copy()
        best_cost = self.sop.total_cost(best_solution)
        improved = True
        
        
        max_passes = 3
        current_pass = 0
        
        while improved and current_pass < max_passes:
            improved = False
            current_pass += 1
            
            
            for i in range(len(best_solution) - 1):
                for j in range(i + 1, len(best_solution)):
                    candidate = best_solution.copy()
                    candidate[i], candidate[j] = candidate[j], candidate[i]
                    
                    if self.sop.is_valid(candidate):
                        candidate_cost = self.sop.total_cost(candidate)
                        if candidate_cost < best_cost:
                            best_solution = candidate
                            best_cost = candidate_cost
                            improved = True
                            break
                
                if improved:
                    break
            
            if improved:
                continue
                
            
            for i in range(len(best_solution)):
                for j in range(len(best_solution)):
                    if i != j:
                        candidate = best_solution.copy()
                        node = candidate.pop(i)
                        candidate.insert(j, node)
                        
                        if self.sop.is_valid(candidate):
                            candidate_cost = self.sop.total_cost(candidate)
                            if candidate_cost < best_cost:
                                best_solution = candidate
                                best_cost = candidate_cost
                                improved = True
                                break
                
                if improved:
                    break
            
            if improved:
                continue
                
            
            
            max_3opt_attempts = 20
            for _ in range(max_3opt_attempts):
                i, j, k = sorted(random.sample(range(len(best_solution)), 3))
                
                
                candidate = best_solution[:i+1] + best_solution[j:k+1] + best_solution[i+1:j] + best_solution[k+1:]
                
                if self.sop.is_valid(candidate):
                    candidate_cost = self.sop.total_cost(candidate)
                    if candidate_cost < best_cost:
                        best_solution = candidate
                        best_cost = candidate_cost
                        improved = True
                        break
        
        return best_solution



def solve_instance(file_path, ants=50, iterations=100, verbose=False):

    sop = SOP(file_path)
    solver = AntColonySolver(sop, ants=ants, iterations=iterations)
    result = solver.solve(verbose=verbose)
    return result


def process_all_instances(data_dir="ACO-SOP/data/", 
                          instance_names=None, 
                          ants=50,
                          iterations=100,
                          parallel=False,
                          verbose=True):
    if instance_names is None:
        instance_names = ["ESC07", "ESC11", "ESC12", "ESC25", "ESC47", "ESC63", "ESC78"]
    
    file_paths = [os.path.join(data_dir, f"{name}.sop") for name in instance_names]
    
    for path in file_paths:
        if not os.path.exists(path):
            print(f"Warning: File {path} not found!")
    
    file_paths = [path for path in file_paths if os.path.exists(path)]
    
    if not file_paths:
        raise FileNotFoundError("No valid SOP files found!")
    
    results = {}
    
    if parallel and len(file_paths) > 1:
        print(f"Processing {len(file_paths)} instances in parallel...")
        with multiprocessing.Pool() as pool:
            args = [(path, ants, iterations, False) for path in file_paths]
            parallel_results = pool.starmap(solve_instance, args)
            
            for result in parallel_results:
                results[result['instance']] = result
    else:
        for path in file_paths:
            if verbose:
                print(f"\n{'='*50}")
                print(f"Processing {os.path.basename(path)}...")
                print(f"{'='*50}")
            
            result = solve_instance(path, ants, iterations, verbose)
            results[result['instance']] = result
    
    if verbose:
        print("\n" + "="*70)
        print("Summary of Results:")
        print("="*70)
        print(f"{'Instance':<10} | {'Cost':>10} | {'Time (s)':>10} | {'Iterations':>12}")
        print("-"*70)
        for name, result in results.items():
            print(f"{name:<10} | {result['cost']:>10.2f} | {result['time']:>10.2f} | {result['iterations']:>12}")
    
    return results


def save_results(results, output_dir="resultsACO"):

    os.makedirs(output_dir, exist_ok=True)

    summary = []
    for name, res in results.items():
        summary.append({
            "instance":   name,
            "cost":       res["cost"],
            "time":       res["time"],
            "iterations": res["iterations"]
        })
    pd.DataFrame(summary).to_csv(
        os.path.join(output_dir, "aco_summary.csv"), index=False
    )

    for name, res in results.items():
        with open(os.path.join(output_dir, f"{name}.sop_solution.txt"), "w") as f:
            f.write(f"Instance: {name}\n")
            f.write(f"Path: {res['solution']}\n")
            f.write(f"Cost: {res['cost']}\n")
            f.write(f"Time: {res['time']:.2f} sec\n")

        pd.DataFrame(res["stats"]).to_csv(
            os.path.join(output_dir, f"{name}.sop_stats.csv"), index=False
        )

    plt.figure(figsize=(10,6))
    for name, res in results.items():
        df = pd.DataFrame(res["stats"])
        plt.plot(df["iteration"], df["best_cost"], label=name)
    plt.xlabel("Iteration")
    plt.ylabel("Best Cost")
    plt.title("ACO Convergence")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "aco_convergence_plot.png"))
    plt.close()

    print(f"All files written to ./{output_dir}/")


if __name__ == "__main__":
    instances = ["ESC07", "ESC11", "ESC12", "ESC25", "ESC47", "ESC63", "ESC78"]
    results = process_all_instances(
        data_dir="../data",
        instance_names=instances,
        ants=50,
        iterations=100,
        parallel=False,
        verbose=True
    )
    
    save_results(results, output_dir="resultsACO")


Processing ESC07.sop...
Iteration 0/100, Best cost: 2125, Alpha: 1.00, Beta: 2.50, Q0: 0.90
Iteration 10/100, Best cost: 2125, Alpha: 0.90, Beta: 2.75, Q0: 0.81
Iteration 20/100, Best cost: 2125, Alpha: 0.90, Beta: 2.75, Q0: 0.81
Stagnation detected at iteration 25 - diversifying search
Iteration 30/100, Best cost: 2125, Alpha: 0.90, Beta: 2.75, Q0: 0.81
Iteration 40/100, Best cost: 2125, Alpha: 0.90, Beta: 2.75, Q0: 0.81
Stagnation detected at iteration 50 - diversifying search
Iteration 50/100, Best cost: 2125, Alpha: 1.00, Beta: 2.50, Q0: 0.90
Early stopping at iteration 51: no improvement for 51 iterations

Solution for ESC07.sop:
Path: [1, 2, 5, 8, 3, 7, 6, 4, 9]
Cost: 2125
Time: 0.33 seconds

Processing ESC11.sop...
Iteration 0/100, Best cost: 2543, Alpha: 1.00, Beta: 2.50, Q0: 0.90
Iteration 10/100, Best cost: 2509, Alpha: 0.90, Beta: 2.75, Q0: 0.81
Iteration 20/100, Best cost: 2495, Alpha: 0.90, Beta: 2.75, Q0: 0.81
Iteration 30/100, Best cost: 2352, Alpha: 1.00, Beta: 2.50, Q